# Custom types and modules
Now that we skimmed through the basic language features, let's introduce classes and modules.
Classes provide an abstraction mechanism to create custom object holding certain associated values and operations.
Modules are essentially a way to bundle and distribute code that can be reused.
We start by introducing the basic concepts and language syntax for classes and modules, then the rest of the tutorial is built around a toy project that will be developed and extended as
a set of exercises and step-by-step examples. We will develop a client-server chat service that we use as a way to show how to think about framing a problem and solving it with python code.

## Classes and objects
An class represents the description of a data type. Every type in python has a corresponding (possibly built-in) class that describes the set of operations that can be performed on the type.
The class is then used to create instances of a type, just like we created dictionaries ad lists before. An instance of a class is usually referred to as an object.

In [ ]:
a_dict = dict() # create instance of dict class.

print(type({}))
print(type([]))

Now this gives us the ability to create custom types that have whatever behaviour we desire. Generally speaking a class is useful to:
- Encapsulate implementation details behind an interface, so that the rest of the program does not need to be aware of how some operation is performed.
- Partition the responsibilility of handling a task within the program.
- Abstract operations on different kinds of objects and present an uniform interface to the rest of the program.

If this sounds very generic, it is. These concepts will hopefully become clearer as we start our toy project.

The way we go about creating a class, is by using the `class` keyword.

In [ ]:
class MyClass:
    pass

my_instance = MyClass()
print(my_instance)

We can define functions associated to objects of our class. When we do so, the first argument to the function is always implicitly set to a reference to the instance on which the function operates, the argument is generally called `self`.
Note that the `self` parameter is implicit and does not need to be given when calling the function on the `my_instance` object, this is because it would be redundant, as the value of `self` is the object on which the function is called, and it is always the one at the left-hand side of the `.` operator.

A function defined by a class is called a *method* of that class.

In [ ]:
class MyClass:
    def do_something(self):
        print("MyClass instance is doing something")
        
my_instance = MyClass()
my_instance.do_something()

There is a set of special function names beginning and ending with a duble underscore `__something__`, these are used to mark functions that have special meaning in the python language.
One of these magic functions is the *constructor*, that is the function called to initialize the new class instance when it is created.

In [ ]:
class MyClass:
    def __init__(self):
        print("I am the constructor!")
    
    def do_something(self):
        print("Doing something")
        
my_instance = MyClass() # this invokes the constructor
my_instance.do_something()

More of these magic methods can be found on the python [documentation](https://docs.python.org/3/reference/datamodel.html).

Instances of classes can also keep state, in the form of variables associated with an object. These member variables are usually called *properties* or *attributes* of an object and are accessed via the `.` operator on the object.
Attributes are usually defined and initialized in the constructor of the class, this will ensure that the name is always defined on new instances. However you can set new attributes on an object just by assigning to them.

In [ ]:
class MyClass:
    def __init__(self):
        self.my_property = 150
        print("I am the constructor!")
        
my_instance = MyClass() # this invokes the constructor

# hasattr() is a built-in function that checks whether an attribute with the given name exists on the object
print("check for my_property:", hasattr(my_instance, "my_property"))
print("check for other_thing:", hasattr(my_instance, "other_thing"))

print("my property is:", my_instance.my_property)
# print(my_instance.other_thing) # this will fail because my_instance does not yet have any property named other_thing

# now we create the new attribute
my_instance.other_thing = "something else"
print("new attribute created:", my_instance.other_thing)

You will notice that methods associated to a given objects are also attributes, and as such can be read and written to. Calling the function is in reality just applying the `()` call operator on the content of the attribute with the function name.
Now methods are not just plain functions shoved into an attribute, this is because they are bound to the class instance, meaning that the `self` parameter will be managed for you.

Now we want to have the class do something useful, let's say we want to create a class representing a point in 2D space.

In [ ]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def add(self, other):
        self.x += other.x
        self.y += other.y
        
p = Point(10, 10)
q = Point(1, 2)
p.add(q)
print(p.x, p.y)

In [ ]:
################################# EXPERIMENT #################################
# Modify the point class to do the following:
# 1. Make the constructor x and y arguments optional, and initialize them to 0 if they are not given
# 2. Add a new method to subtract a point from another
# 3. Add a pretty print method that converts a Point object to string suitable for printing and converting to a string using the str() built-in. (hint: use the magic __str__() method)

### Attribute visibility
Another important topic regarding classes is the fact that they are used to hide implementation details from the user of the class.
Let's continue the example using the Point class. We can think of many ways in which we can store the point coordinates, for instance we could store them as attributes as above, or we may prefer to store them in a list.
This decision however affects how the use of the class accesses the coordinates, this is because we chose to make the object state concerning the coordinates visible from outside the class (or **public**).
Note that in this specific case it may seem trivial, but it can easily impose unnecessary constraints and complication because now other parts of the code are _aware_ and _rely_ upon the internal state of the object being this way.
If we wanted to switch from storing coordinates as attributes to storing them into an array, we would have to modify every single use in the code.

In [ ]:
class PointWithAttr:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class PointWithList:
    def __init__(self, x, y):
        self.coords = [x, y]
        
# Note that the two implementations are not compatible, and yet they represent the exact same data
p = PointWithAttr(1, 2)
q = PointWithList(3, 4)
print(p.x, p.y)
print(q.coords[0], q.coords[1])

The idea of making internal implementation details invisible from the user of the class is achieved by making the relevant attributes **private** members of the class.
This means that they are removed from the public interface of the class and whoever uses instances of the class can not access them directly, but only via class methods.
In python **private** class properties and methods are somewhat awkward, as the language does not have proper `private` keyword, instead there are two ways in which an attribute is treated as private:
1. Attributes beginning with a single underscore (as in `_foo`) are treated as private **by convention**, meaning that if you see one you can still access it but the designer of the code intended to treat it as an implementation detail an it may change without any notice in different versions of the code, breaking your code. This is the way most people handle private members and is widely agreed upon.
2. Attributes beginning with a double underscore and at most one trailing underscore have their name automatically prefixed by something (which depends on the python implementation, usually the class name), so that accessing the name from outside the class will not find the attribute with the original name.

In [ ]:
class Point:
    def __init__(self, x, y):
        self._coords = {"x": x, "y": y}
        
    def get_x(self):
        return self._coords["x"]
    def set_x(self, v):
        self._coords["x"] = v
        
    def get_y(self):
        return self._coords["y"]
    def set_y(self, v):
        self._coords["y"] = v
        
p = Point(10, 20)
print("My point (", p.get_x(), p.get_y(), ")")

In [ ]:
################################# EXPERIMENT #################################
# Change the Point class back to using two private x and y attributes to store the coordinates
# **without** changing the interface of the class (meaning the set of publicly visible functions used to access the attributes)
# so that the constructor and print statement below remain working.

### Inheritance

The idea of inheritance between two classes establishes an *is-a* relationship between them, where a child class *inherits* the methods and attributes of a parent class.
For the sake of simplicity, let's make a class to represent animals. Now we want to have cats and dogs, each of them *is an animal* so it makes sense for the Cat and Dog classes to inherit from Animal and share its interface, meaning that they both will have a speed and will have a name, but only cats will `meaow()` and only dogs will `bark()`.

In [ ]:
class Animal:
    def __init__(self, speed):
        self.speed = speed
        
    def get_name(self):
        return "I don't have a name :("
        
    def move(self):
        print("Moving at ", self.speed)
        
class Cat(Animal):
    def __init__(self):
        # super() provides a reference to the next parent class in the
        # inheritance chain (well graph, when considering multiple inheritance)
        super().__init__(5)
        
    # Note that this method redefines the get_name function and hides the parent's implementation
    def get_name(self):
        return "I am a cat"
    
    def meaow(self):
        print("Meaow!")
        
class Dog(Animal):
    def __init__(self):
        super().__init__(4)
    
    def get_name(self):
        return "I am a dog"
    
    def bark(self):
        print("Woof!")
        
cat = Cat()
dog = Dog()
animals = [cat, dog]
for a in animals:
    print(a.get_name())
    # Note that move() here will call Animal.move, which is inherited by both Cat and Dog
    a.move()

## Challenge #1 - Geometry shapes
Write a program that allows you to select a shape, enter its measures and the color. It will output the perimeter and area as a result.
The program should support the following shapes: circle, square, rectangle.


In [ ]:
# Try to implement it here or as a separate script to run manually on the terminal!

In [ ]:
# Possible solution

def get_int(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid value")

class Shape:
    def __init__(self):
        self.color = input("Enter color")
        
    def area(self):
        return self._calc_area()
        
    def perimeter(self):
        return self._calc_perimeter()


class Circle(Shape):
    def __init__(self):
        super().__init__()
        self.radius = get_int("Enter radius")
                
    def _calc_perimeter(self):
        return 3.14 * 2 * self.radius
    
    def _calc_area(self):
        return 3.14 * self.radius**2
    
    def __str__(self):
        return "circle"


class Rectangle(Shape):
    def __init__(self):
        super().__init__()
        self.width = None
        self.height = None
        self._ask_sides()
        
    def _ask_sides(self):
        self.width = get_int("Enter width")
        self.height = get_int("Enter height")
        
    def _calc_perimeter(self):
        return 2 * (self.width + self.height)
    
    def _calc_area(self):
        return self.width * self.height
    
    def __str__(self):
        return "rectangle"


class Square(Rectangle):        
    def _ask_sides(self):
        self.width = self.height = get_int("Enter side")
        
    def __str__(self):
        return "square"


while True:
    shape_name = input("Insert shape name or blank to quit")
    if shape_name == "":
        break
    if shape_name == "circle":
        shape = Circle()
    elif shape_name == "square":
        shape = Square()
    elif shape_name == "rectangle":
        shape = Rectangle()
    else:
        continue

    print("{} {}: perimeter={} area={}".format(shape.color, shape, shape.perimeter(), shape.area()))
    